# SQL

¿Qué es SQL? Son las siglas de "Structured Query Language" y se utiliza para administrar y consultar tablas almacenadas en un Sistema Gestor de Bases de Datos Relacional. 

# Conexión al laboratorio SQL

In [4]:
!pip install sqlalchemy
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 129.4 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
from sqlalchemy import create_engine
import sqlalchemy
import mysql.connector as mysql
import pandas as pd

# Creamos la conexión

In [ ]:
# Parámetros de conexión
user     = 'root'
password = ''
#db = ''
sqlServer = 'localhost' # debería ser IP o DNS (Domain Name System) del servidor de la base de datos

# Creamos string de conexión
connStr = "mysql+mysqlconnector://" + user + ":" + password + "@" + sqlServer # +  "/"  + db

# print de string de conexión
print (connStr)

# Conectamos con el servidor
conn = create_engine(connStr).connect() 

# Creamos nuestra BD

In [ ]:
# Crea una BD con nombre NUCLIO
sSQL = ""
conn.execute(sSQL)

# De otra manera o si ya lo tenemos creado:
# conn.execute("DROP DATABASE nombre") # Para eliminar la base de datos si ya existe
# conn.execute("CREATE DATABASE nombre") # Par crear la base de datos

## Mostramos el resultado de la ejecución

In [ ]:
# Comando para ver todas las bases de datos disponibles en el servidor
dbs = conn.execute("SHOW DATABASES")

In [ ]:
# Número de filas en el resultado
dbs.rowcount

In [ ]:
# Mostramos el resultado utilizando la función .fetchall()
print(dbs.fetchall())

In [ ]:
# Especificamos que queremos usar la base de datos NUCLIO
conn.execute("USE NUCLIO")

¡¡Intentemos esto en la consola!!

# Creación de tablas

Primero podemos ver que no hay ninguna tabla creada en nuestra base de datos

In [ ]:
# Mostramos las tablas disponibles en nuestra BD
tables = conn.execute("SHOW TABLES")
tables.fetchall()

La creación de una tabla se realiza mediante la sentencia CREATE TABLE. La versión básica de esta sentencia incluye la definición de los atributos y sus tipos de datos correspondientes, el valor por defecto que toma un atributo cuando no se especifica su valor al insertar una nueva tupla (cláusula DEFAULT), así como las claves primaria y externas.

Cabe destacar que cuando la clave primaria o la clave externa está formada por un solo atributo, las palabras reservadas PRIMARY KEY y UNIQUE, respectivamente, se podrán incluir a continuación de la definición del atributo correspondiente (ejemplo 1) (https://dev.mysql.com/doc/refman/8.0/en/create-table.html)

In [ ]:
# Sintaxis de creación de una tabla
# 
# CREATE TABLE nombre-tabla (
#     nombre-atributo1 data-type-1 [DEFAULT expr],
#     nombre-atributo2 data-type-2 [DEFAULT expr],
#     [PRIMARY KEY (nombre-atributo1, nombre-atributo2,...),]
#     [UNIQUE (nombre-atributo1, nombre-atributo2,...),]
#     [FOREIGN KEY (nombre-atributo1, nombre-atributo2,...)
#         REFERENCES nombre-tabla(nombre-atributo,...),]
#     [CHECK(condicion)]
# 
# );


# Ejemplo 1
# 
# CREATE TABLE nombre-tabla1(
#     nombre-atributo1 data-type-1 PRIMARY KEY,
#     nombre-atributo2 data-type-2 UNIQUE,
# );

Supongamos que somos propietarios de una empresa y tenemos información de los trabajadores. Tenemos varias formas de crear tablas:

In [ ]:
# Creamos una tabla con información de trabajadores, llamandola plantilla
sSQL = """
    CREATE TABLE plantilla (
        dni CHAR(8),
        nombre VARCHAR(15) DEFAULT 'sin nombre',
        estadocivil VARCHAR(10)
            CHECK (estadocivil IN ('soltero','casado','divorciado','viudo')),
        fechaalta DATE NOT NULL,
        PRIMARY KEY (dni)
    )

"""

conn.execute(sSQL)

# Veamos que para crear la clave primaria, este código valdría también
sSQL1 = """
    CREATE TABLE plantilla_v1 (
        dni CHAR(8) PRIMARY KEY,
        nombre VARCHAR(15) DEFAULT 'sin nombre',
        estadocivil VARCHAR(10)
            CHECK (estadocivil IN ('soltero','casado','divorciado','viudo')),
        fechaalta DATE NOT NULL
    )

"""

conn.execute(sSQL1)

# Podemos poner los checks al final
sSQL2 = """
    CREATE TABLE plantilla_v2 (
        dni CHAR(8) PRIMARY KEY,
        nombre VARCHAR(15) DEFAULT 'sin nombre',
        estadocivil VARCHAR(10),
        fechaalta DATE NOT NULL,
        CHECK (estadocivil IN ('soltero','casado','divorciado','viudo'))
    )

"""

conn.execute(sSQL2)

# Podemos poner los checks y claves al final
sSQL3 = """
    CREATE TABLE plantilla_v3 (
        dni CHAR(8),
        nombre VARCHAR(15) DEFAULT 'sin nombre',
        estadocivil VARCHAR(10),
        fechaalta DATE NOT NULL,
        PRIMARY KEY (dni),
        CHECK (estadocivil IN ('soltero','casado','divorciado','viudo'))
    )

"""

conn.execute(sSQL3)

In [ ]:
# Nos dará la información: Field, Type, Null, Key, Default, Extra.
conn.execute('SHOW COLUMNS FROM plantilla').fetchall()

In [ ]:
# Nos dará la información: Field, Type, Null, Key, Default, Extra
conn.execute('SHOW COLUMNS FROM plantilla_v1').fetchall()

In [ ]:
# Nos dará la información: Field, Type, Null, Key, Default, Extra
conn.execute('SHOW COLUMNS FROM plantilla_v2').fetchall()

In [ ]:
# Nos dará la información: Field, Type, Null, Key, Default, Extra
conn.execute('SHOW COLUMNS FROM plantilla_v3').fetchall()

Podemos ver esta información en la consola

In [ ]:
# Podemos almacenar esta información en un pandas dataframe, ¿cómo?


In [ ]:
# Podemos obtener la misma información con el comando DESC
conn.execute('DESC plantilla').fetchall()

In [ ]:
# ¿Cómo veríamos esta misma información en un pandas dataframe?


In [ ]:
# Supongamos que ahora queremos almacenar la relación entre jefes y empleados a partir de la tabla plantilla, podremos crear otra tabla con la siguiente estructura:

sSQL = """

    CREATE TABLE serjefe(
        dnijefe CHAR(8),
        dnitrabajador CHAR(8),
        PRIMARY KEY (dnitrabajador),
        FOREIGN KEY (dnijefe) REFERENCES plantilla(dni),
        FOREIGN KEY (dnitrabajador) REFERENCES plantilla(dni)
    )


"""

conn.execute(sSQL)

> ¿Por qué referenciamos dnitrabajador como PRIMARY KEY y FOREIGN KEY? ¿No sería suficiente con la FOREIGN KEY?

In [ ]:
# Nos dará la información: Field, Type, Null, Key, Default, Extra
conn.execute('SHOW COLUMNS FROM serjefe').fetchall()

In [ ]:
# En pandas dataframe


In [ ]:
# Podemos obtener la misma información con el comando DESC
conn.execute('DESC serjefe').fetchall()

In [ ]:
# En pandas dataframe


In [ ]:
# Volvemos a identificar las tablas disponibles
tables = conn.execute("SHOW TABLES")
tables.fetchall()

In [ ]:
# ¿Se nos ocurre alguna forma de hacer un print de las tablas una por una a través de un bucle for?


## Cargamos tabla en la BD desde DataFrame

Veamos como podemos cargar tablas en nuestra BBDD de diferentes formas y orígenes:

In [ ]:
# Supongamos que tenemos información relacionada con algunos clientes
customers={'cust_id':['a','b','c','d'],
           'name':['Alice','Bob','Carlos','Dieter'],
           'country':['us','ca','mx','de']}

# Pasa esta información a un pandas.DataFrame llamado df_customers


In [ ]:
# Podemos escribir en nuestra base de datos a través de: df.to_sql(name, if_exists, con)


In [ ]:
# Sube esta información como tabla a nuestra BD
orders={'order_id':[1,2,3,4,5],
        'cust_id':['a','c','a','b','z'],
        'total':[1539,1871,6352,1456,2137]}



In [ ]:
# Volvemos a identificar las tablas disponibles
tables = conn.execute("SHOW TABLES")
tables.fetchall()

## Creación a partir de un archivo sql

In [ ]:
# Leemos el archivo proveedor.sql
with open('files/proveedor.sql', 'r') as file:
    sql_script = file.read()

In [ ]:
# Ejecuta el script que acabamos de leer


In [ ]:
conn.execute('SHOW TABLES').fetchall()

In [ ]:
conn.execute('DESC proveedor').fetchall()

## Creación a partir de un fichero csv

In [ ]:
# Primero leemos el archivo
products=pd.read_csv("files/products.csv", sep=',')
products

In [ ]:
# Utilizamos el dataframe para escribirlo en nuestra BBDD
# Es exactamente igual que con un pandas DataFrame
# Crea la tabla products a partir del dataframe products


In [ ]:
# vemos que la tabla se ha añadido correctamente
conn.execute("SHOW TABLES").fetchall()

In [ ]:
# Vamos a abrir un archivo con información sobre nuestras ventas
sales=pd.read_csv("files/sales.csv", sep=',')
sales

In [ ]:
# utilizamos el dataframe para escribirlo en nuestra BBDD


In [ ]:
# vemos que la tabla se ha añadido correctamente
conn.execute("SHOW TABLES").fetchall()

¡¡Check de que todas las tablas aparecen en nuestra base de datos en la consola!!

## Creación a partir de un fichero xlsx

Ojo! hay que tener cuidado cuando trabajamos con ficheros .xlsx

In [ ]:
# pandas nos permite leer excels
sellers=pd.read_excel("files/Sellers.xlsx")
sellers

In [ ]:
# Escribe la tabla sellers en la BD


In [ ]:
# vemos que la tabla se ha añadido correctamente
conn.execute("SHOW TABLES").fetchall()

## Creación a partir de un fichero html

In [ ]:
# pandas nos permite leer ficheros html
clients=pd.read_html("files/clients.html")
clients

In [ ]:
clients[0]

In [ ]:
# Crea la tabla clients a partir de la información del fichero html


In [ ]:
conn.execute("SHOW TABLES").fetchall()

Check de que vamos alineados en la consola

# Eliminación de tablas

Al eliminar las tablas, además eliminamos todas sus tuplas y liberamos espacio

In [ ]:
# Se puede eliminar una tabla con todas las tuplas que contiene, liberando el espacio con la sentencia:
# DROP TABLE nombre-tabla (https://dev.mysql.com/doc/refman/8.0/en/drop-table.html)
# Borra la tabla serjefe


In [ ]:
# Volvemos a identificar las tablas disponibles
tables = conn.execute("SHOW TABLES")
tables.fetchall()

¿Qué ocurre cuando eliminamos una tabla a la que hace referencia con una clave externa a otra tabla?

In [ ]:
sSQL = """

    CREATE TABLE serjefe_v2(
        dnijefe CHAR(8),
        dnitrabajador CHAR(8),
        PRIMARY KEY (dnitrabajador),
        FOREIGN KEY (dnijefe) REFERENCES plantilla_v2(dni),
        FOREIGN KEY (dnitrabajador) REFERENCES plantilla_v2(dni)
    )


"""

conn.execute(sSQL)

In [ ]:
# Borra la tabla plantilla_v2


# Modificación de tablas

Para una tabla existente, podemos utilizar la sentencia ALTER TABLE para modificar su estructura. Por ejemplo, podemos añadirle una nueva columna, modificar la definición o las restricciones de algunos de sus atributos, o bien, eliminar algún atributo. Para todo el detalle podemos consultar la sintaxis completa en https://dev.mysql.com/doc/refman/8.0/en/alter-table.html

In [ ]:
# Por ejemplo, vamos a añadir una columna llamada fechabaja de tipo DATE a la tabla plantilla
sSQL = """


"""

conn.execute(sSQL)

In [ ]:
# Nos dará la información: Field, Type, Null, Key, Default, Extra
conn.execute('SHOW COLUMNS FROM plantilla').fetchall()

In [ ]:
# Podemos obtener la misma información con el comando DESC
conn.execute('DESC plantilla').fetchall()

In [ ]:
# Podemos además añadir restricciones con una nueva columna: añadimos una columna a plantilla de tipo entero que no sea nulo y llamala entero_no_nulo
sSQL = """


"""

conn.execute(sSQL)

In [ ]:
# Mostramos la información de las columas en un pandas dataframe (SHOW COLUMNS FROM table)


In [ ]:
# Crea una nueva columna a plantilla que se llame escuela, de tipo VARCHAR(8) que tenga como valor por defecto "nuclio"
sSQL = """



"""

conn.execute(sSQL)

In [ ]:
# Mostramos la información de las columas en un pandas dataframe (DESC o SHOW COLUMNS)


In [ ]:
# Crea una nueva columna en plantilla llamada rango de tipo entero que se encuentre en el rango [0,10]
sSQL = """



"""

conn.execute(sSQL)

In [ ]:
# Checkea la información


Al utilizar ALTER TABLE también podemos **modificar el tipo de dato** de una columna:

In [ ]:
# DECIMAL(digitos, decimales)
sSQL = """

ALTER TABLE plantilla 
MODIFY COLUMN rango DECIMAL(10,2)


"""

conn.execute(sSQL)

Al utilizar ALTER TABLE también podemos **renombrar** columnas:

In [ ]:
sSQL = """

ALTER TABLE plantilla 
RENAME entero_no_nulo TO entero


"""

conn.execute(sSQL)

Al utilizar ALTER TABLE también podemos **borrar** columnas:

In [ ]:
sSQL = """

ALTER TABLE plantilla 
DROP COLUMN rango


"""

conn.execute(sSQL)

También podemos añadir todas las restricciones que queramos, así como crear y borrar índices y restricciones de claves externas

# Renombrar tablas

Podemos modificar el nombre de las tablas utilizando RENAME (https://dev.mysql.com/doc/refman/8.0/en/rename-table.html)

In [ ]:
# Modifica el nombre de la tabla serjefe_v2 a serjefe_v3. Comprueba el resultado


In [ ]:
# Vuelve a llamarla serjefe_v2. Comprueba el resultado


> ¿Qué ocurrirá con las dependencias en las claves foráneas?

In [ ]:
conn.execute("SHOW CREATE TABLE serjefe_v2")

In [ ]:
# Modifica el nombre de plantilla_v2 a nueva_plantilla y comprueba el resultado.


# Particiones

Las particiones permiten dividir una tabla en **bloques** o **segmentos** de datos, lo que hace más eficiente el acceso, almacenamiento y manejo de los mismos. Cada partición puede ser almacenada en un archivo físico separado, lo que permite que solo una porción de la tabla sea escaneada o procesada durante ciertas consultas. Esto es particularmente útil cuando la tabla tiene millones o incluso miles de millones de registros.

Tenemos varios tipos:

- Por **rango**: los datos se dividen en rango de valores (partir una tabla por año o meses)
- Por **lista**: en función de valores específico (partir una tabla por el país o región de los clientes)
- Por **hash**: los datos se distribuyen entre las particiones de manera aleatoria o uniforme, utilizando una función de hash. Algun ejemplo puede ser cuando se quiere dividir los datos de manera más o menos uniforme cuando no se conoce un criterio claro para hacerlo
- Por **rango-hash o rango-lista**: se combina un criterio por rango con un criterio por lista o hash para manejar datos más complejos

Supongamos que tenemos una tabla ventas que almacena transacciones de varios años, y la mayoría de las consultas buscan ventas recientes. Usar una partición por rango basado en la columna fecha nos ayudará a segmentar los datos por año o mes, mejorando el rendimiento de las consultas que acceden a los datos más recientes.

In [ ]:
# Partición por rango
sSQL = """

CREATE TABLE ventas (
    id INT,
    fecha DATE,
    total DECIMAL(10,2),
    cliente_id INT
) PARTITION BY RANGE(YEAR(fecha)) (
    PARTITION p2022 VALUES LESS THAN (2023),
    PARTITION p2023 VALUES LESS THAN (2024),
    PARTITION p2024 VALUES LESS THAN (2025)
)


"""

In [ ]:
sSQL = "SHOW CREATE TABLE ventas"

- Las ventas del 2022 van a la partición p2022, las del 2023 a p2023, y así sucesivamente.
- Cuando consultamos solo ventas recientes (ej. ventas del año 2023), el motor de la base de datos solo escaneará la partición correspondiente, acelerando la consulta.

Tenemos una tabla clientes con datos de clientes de diferentes países. Podríamos dividir la tabla en particiones basadas en el país de residencia para que las consultas que segmentan por país sean más rápidas

In [ ]:
# Partición por lista
sSQL = """

CREATE TABLE clientes (
    id INT,
    nombre VARCHAR(100),
    pais VARCHAR(50)
) PARTITION BY LIST (pais) (
    PARTITION p_usa VALUES IN ('USA'),
    PARTITION p_canada VALUES IN ('Canada'),
    PARTITION p_mexico VALUES IN ('Mexico'),
    PARTITION p_otro VALUES IN ('UK', 'France', 'Germany', 'Japan')
)


"""

Tenemos una tabla pedidos con millones de registros y no hay un campo obvio para hacer una partición por rango o lista, pero queremos distribuir los datos equitativamente en particiones para mejorar el rendimiento de las consultas

In [ ]:
# Partición por hash
sSQL = """

CREATE TABLE pedidos (
    id INT,
    cliente_id INT,
    fecha DATE,
    total DECIMAL(10, 2)
) PARTITION BY HASH(cliente_id) PARTITIONS 4


"""

Tenemos una tabla transacciones con millones de registros de diferentes países y fechas. Queremos partir los datos por años (rango) y luego dentro de cada año, segmentarlos por país (lista).

In [ ]:
# Partición por rango-lista
sSQL = """

CREATE TABLE transacciones (
    id INT,
    fecha DATE,
    total DECIMAL(10, 2),
    pais VARCHAR(50)
) PARTITION BY RANGE (YEAR(fecha)) SUBPARTITION BY LIST (pais) (
    PARTITION p2022 VALUES LESS THAN (2023) (
        SUBPARTITION usa VALUES IN ('USA'),
        SUBPARTITION canada VALUES IN ('Canada'),
        SUBPARTITION mexico VALUES IN ('Mexico')
    ),
    PARTITION p2023 VALUES LESS THAN (2024) (
        SUBPARTITION usa VALUES IN ('USA'),
        SUBPARTITION canada VALUES IN ('Canada'),
        SUBPARTITION mexico VALUES IN ('Mexico')
    )
)


"""

Aquí:

- Los datos primero se particionan por año (rango).
- Luego, dentro de cada año, se particionan por país (lista).

Esto permite una optimización más granular en consultas que filtran por año y país al mismo tiempo.

# Comentarios

En MySQL, los comentarios se utilizan para documentar el código SQL, facilitar la comprensión del mismo y proporcionar información adicional sobre el propósito o la funcionalidad de una consulta, tabla, columna, procedimiento almacenado u otro elemento de la base de datos. Hay dos tipos principales de comentarios que se pueden utilizar en MySQL (https://www.mysqltutorial.org/mysql-comment/):

- **Comentarios de una línea**: Estos comentarios comienzan con el símbolo -- (dos guiones seguidos) y abarcan el resto de la línea. Todo lo que se escriba después de -- en la misma línea se considera un comentario y no se ejecuta como parte del código SQL.
sql


In [ ]:
sSQL = """

-- Este es un comentario de una línea en MySQL
DESC products; -- Describe la tabla 'products'

"""

- **Comentarios de múltiples líneas**: Estos comentarios pueden abarcar varias líneas de código y se encierran entre /* y */. Todo lo que esté dentro de estos delimitadores se considera un comentario y no se ejecuta como parte del código SQL.

In [ ]:
sSQL = """

/* Este es un comentario de
   múltiples líneas en MySQL */
SELECT * FROM customers;

"""

# Cerramos conexión

In [ ]:
# Cerramos conexión con el servidor
conn.close()